# Install prerequisites

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 119.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.5 MB/s eta 0:00:00


In [ ]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.9 MB/s eta 0:00:00


In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Input the TEST pdf and EMBEDDINGS

In [ ]:
from transformers import BertTokenizer, BertModel
import PyPDF2
import re
import unicodedata
from nltk.corpus import stopwords
import pandas as pd
import torch
import numpy as np

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

text_column = "PDF"
k = 5

stop_words = set(stopwords.words('english'))

In [ ]:
csv_path = "/Lemmatized_Final_Dataset.csv"
df = pd.read_csv(csv_path)

In [ ]:
df.head()

,AUTHOR,TITLES,PDF
0,Aruna Tiwari,A Quantum-Inspired Fuzzy Based Evolutionary.pdf,see discussions ats author ofiles public ation...
1,Aruna Tiwari,A Review of Clustering Techniques and Developm...,review clustering techniques developments amit...
2,Aruna Tiwari,Apache Spark Based Kernelized Fuzzy Clustering...,see discussions ats author ofiles public ation...
3,Aruna Tiwari,Breast cancer diagnosis using Genetically Opti...,breast cancer diagnosis using genetically opti...
4,Aruna Tiwari,Enhanced Cluster Validity Index for the Evalua...,enhanced cluster validity index evaluation opt...


In [ ]:
df.shape

(1211, 3)

# Defining functions for subtasks

In [ ]:
def preprocess_test_document(pdf_path):
    test_document_text = extract_text_from_pdf(pdf_path)
    preprocessed_text = preprocess_text(test_document_text)
    return preprocessed_text

def preprocess_text(text):
    if not isinstance(text, str):
        return ''
    text = text.replace('\n', ' ').replace('\r', ' ')
    text = re.sub('<[^>]*>', '', text)
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    text = re.sub('http[s]?://\S+', '', text)
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = text.split()
    text = [word for word in text if word not in stop_words]
    text = [word for word in text if len(word) > 2]
    text = ' '.join(text)
    return text

def extract_text_from_pdf(pdf_path):
    try:
        with open(pdf_path, 'rb') as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            pdf_text = ''
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                pdf_text += page.extract_text()
    except KeyError as e:
        print(f"Error processing file '{pdf_path}': {e}")
        return ''
    return pdf_text

def preprocess_csv_file(csv_path, text_column):
    data = pd.read_csv(csv_path)
    csv_text_data = data[text_column].tolist()
    preprocessed_texts = [preprocess_text(text) for text in csv_text_data]
    return preprocessed_texts

def encode_documents(documents):
    encoded_texts = tokenizer(documents, padding=True, truncation=True, return_tensors='pt')
    # For more details on padding and truncation https://huggingface.co/docs/transformers/pad_truncation
    return encoded_texts

def generate_embeddings(encoded_texts):
    # Disabling gradient calculation is useful for inference.
    # It will reduce memory consumption for computations that would otherwise
    # have requires_grad=True.
    # https://pytorch.org/docs/stable/generated/torch.no_grad.html
    with torch.no_grad():
        model.eval()
        input_ids = encoded_texts['input_ids']
        attention_mask = encoded_texts['attention_mask']
        output = model(input_ids=input_ids, attention_mask=attention_mask)
        embeddings = output.last_hidden_state[:, 0, :]
        print(output.last_hidden_state.shape)
    return embeddings

def calculate_similarity(test_embedding, csv_embeddings):
    similarity_scores = []
    indices = []
    for i, csv_embedding in enumerate(csv_embeddings):
        similarity =  torch.cosine_similarity(test_embedding, csv_embedding)
        similarity_scores.append(similarity.item())
        indices.append(i)
    return csv_embeddings, similarity_scores, indices

def retrieve_top_k_unique_authors(csv_data, similarity_scores, indices, k):
    # Sort scores and corresponding indices
    sorted_scores_indices = sorted(zip(similarity_scores, indices), key=lambda x: x[0], reverse=True)
    unique_authors = []

    # Loop over sorted scores and indices
    for _, idx in sorted_scores_indices:
        # Get author corresponding to index
        author = csv_data.iloc[idx]['AUTHOR']
        # Append author to unique_authors list if it's not there yet
        if author not in unique_authors:
            unique_authors.append(author)
            # Break the loop if we have found k unique authors
            if len(unique_authors) == k:
                break

    return unique_authors

In [ ]:
# Preprocess the test PDF document
#Test pdf for evaluation
pdf_path = "/Supervised Heterogeneous Domain Adaptation via Random Forests.pdf"
preprocessed_test_document = preprocess_test_document(pdf_path)

In [ ]:
# Encode the test document and CSV document text
encoded_test_document = encode_documents([preprocessed_test_document])

In [ ]:
encoded_test_document['input_ids'].shape

torch.Size([1, 512])

In [ ]:
# Generate embeddings for the test document
test_embedding = generate_embeddings(encoded_test_document)

# Save the test embedding to a file
np.save("test_embedding.npy", test_embedding.numpy())

# Load the test embedding from the file
test_embedding = torch.from_numpy(np.load("test_embedding.npy"))

torch.Size([1, 512, 768])


In [ ]:
csv_embeddings = torch.from_numpy(np.load("/csv_embeddings_submission.npy"))

In [ ]:
csv_embeddings.shape #Embedding for each document

torch.Size([623, 768])

In [ ]:
# Calculate similarity between the test document and CSV documents
csv_embeddings, similarity_scores, indices = calculate_similarity(test_embedding, csv_embeddings)

# Load the CSV data
csv_data = pd.read_csv(csv_path)

# Retrieve top-k unique authors
top_k_authors = retrieve_top_k_unique_authors(csv_data, similarity_scores, indices, k)

# Print the top-k authors
print(top_k_authors)

['Dilip Singh Sisodia', 'dipanjan roy', 'Nishchal K. Verma', 'Krishna Asawa', 'Tandra Pal']


In [ ]:
# Function to calculate similarity
def calculate_similarity_and_get_most_similar(test_embedding, csv_embeddings, csv_data):
    highest_similarity = -1
    most_similar_index = -1
    similarity_scores = []
    for i, csv_embedding in enumerate(csv_embeddings):
        similarity = torch.nn.functional.cosine_similarity(test_embedding, csv_embedding.unsqueeze(0))
        similarity_scores.append(similarity.item())
        if similarity.item() > highest_similarity:
            highest_similarity = similarity.item()
            most_similar_index = i
    author_name = csv_data.loc[most_similar_index, 'AUTHOR']  # assuming your author column is named 'AUTHOR'
    return highest_similarity * 100, most_similar_index, author_name, [score * 100 for score in similarity_scores]

In [ ]:
highest_similarity, most_similar_index, author_name, similarity_scores = calculate_similarity_and_get_most_similar(test_embedding, csv_embeddings, csv_data)

print(f"The most similar document is at index {most_similar_index} with a similarity of {highest_similarity}%.")
print(f"The author of the most similar document is {author_name}.")

The most similar document is at index 445 with a similarity of 100.0%.
The author of the most similar document is Dilip Singh Sisodia.


In [ ]:
def top_k_accuracy(y_true, y_pred, k=1):
    """
    Compute the top-k accuracy based on word presence in predictions.

    Parameters:
    y_true (str): True label (actual author).
    y_pred (list): List of predicted authors for the test PDFs.
                   The shape of y_pred should be (num_PDFs, k).
    k (int): The value of k for top-k accuracy. Default is 1 (top-1 accuracy).

    Returns:
    top_k_acc (float): The top-k accuracy value.
    """
    if k <= 0 or not isinstance(k, int):
        raise ValueError("k must be a positive integer.")

    num_samples = len(y_pred)
    num_correct = 0

    for i in range(num_samples):
        if y_true in y_pred[i][:k]:
            num_correct += 1

    top_k_acc = num_correct / num_samples
    return top_k_acc

In [ ]:
# Top-3 accuracy with an example
top_k_accuracy('Sanatan',[['Sanatan','Arash','Abhinav'],['Arash','Abhinav','Vartika'],['Arash','Sanatan','Abhinav']],3)

0.6666666666666666

In [ ]:
# Top-1 accuracy (Matching with the first prediction only)
top_k_accuracy('Sanatan',[['Sanatan','Arash','Abhinav'],['Arash','Abhinav','Vartika'],['Arash','Sanatan','Abhinav']],1)

0.3333333333333333

## Some pointers for unbiased evaluation
> You can split the given dataset into train and test, and evaluate the performance of your models with Top-k accuracy **for different values of k**.


* When dividing the data, make sure that at least one PDF file from each author appears in both the training and testing sets. Exclude authors for whom there is only one PDF available in the dataset.

* You can average the performance over multiple runs.

* You can think of aggregating and comparing documents as a whole (per author).



# Closer Look at what the tokenizer does

In [ ]:
sample = 'Let\'s take a deep dive in NLP!'
encoding = tokenizer.encode(sample,  padding=True, truncation=True)
print(encoding)

[101, 2292, 1005, 1055, 2202, 1037, 2784, 11529, 1999, 17953, 2361, 999, 102]


In [ ]:
print(tokenizer.convert_ids_to_tokens(encoding))

['[CLS]', 'let', "'", 's', 'take', 'a', 'deep', 'dive', 'in', 'nl', '##p', '!', '[SEP]']


In [ ]:
enc = tokenizer([sample], padding=True, truncation=False, return_tensors='pt')

In [ ]:
enc

{'input_ids': tensor([[  101,  2292,  1005,  1055,  2202,  1037,  2784, 11529,  1999, 17953,
          2361,   999,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}